**Desafío - Transformación y manipulación de
datos (Parte II)**

Benjamin Corvalan P.

**Descripción**


La empresa ADL se dedica a la venta de productos a nivel regional en los Estados Unidos, con equipos de ventas que operan en diferentes estados.

La compañía ha establecido un sistema de comisiones para incentivar a sus equipos de ventas a preferir la venta de productos de alto margen antes que los de bajo margen de utilidad.


Como Analista de Datos, le solicitan analizar información sobre las ventas realizadas por los equipos de ventas de ADL, incluyendo detalles sobre los productos vendidos, los clientes, las fechas de los pedidos y los plazos de entrega.


In [1]:
import pandas as pd
import numpy as np

1. Carga los datos de cada pestaña del archivo US_Regional_Sales_Data.xlsx en un
DataFrame independiente. (hint: Puedes obtener los nombres de las hojas usando
pd.ExcelFile(archivo).sheet_names)


In [2]:
pd.ExcelFile('US_Regional_Sales_Data.xlsx').sheet_names

['Sales Orders Sheet',
 'Customers Sheet',
 'Store Locations Sheet',
 'Products Sheet',
 'Sales Team Sheet']

In [9]:
sales_df = pd.read_excel("US_Regional_Sales_Data.xlsx", sheet_name='Sales Orders Sheet')
customers_df = pd.read_excel("US_Regional_Sales_Data.xlsx", sheet_name='Customers Sheet')
stores_df = pd.read_excel("US_Regional_Sales_Data.xlsx", sheet_name='Store Locations Sheet')
products_df = pd.read_excel("US_Regional_Sales_Data.xlsx", sheet_name='Products Sheet')
sales_team_df = pd.read_excel("US_Regional_Sales_Data.xlsx", sheet_name='Sales Team Sheet')

#Para validar, cambiamos el nombre del df
customers_df.head()

,_CustomerID,Customer Names
0,1,Avon Corp
1,2,WakeFern
2,3,"Elorac, Corp"
3,4,ETUDE Ltd
4,5,Procter Corp


2. Cruza todos los DataFrames usando validación many_to_one, y guarda el resultado en una nueva variable llamada df_base.

In [17]:
#con esto validamos que existan muchas ventas o registros de ventas pero una sola coincidencia para customer, store, productos o sales team.
df_base = sales_df.merge(customers_df, how='left', on='_CustomerID', validate='m:1')\
                  .merge(stores_df, how='left', on='_StoreID', validate='m:1')\
                  .merge(products_df, how='left', on='_ProductID', validate='m:1')\
                  .merge(sales_team_df, how='left', on='_SalesTeamID', validate='m:1')

print(df_base)



       OrderNumber Sales Channel WarehouseCode ProcuredDate  OrderDate  \
0      SO - 000101      In-Store  WARE-UHY1004   2017-12-31 2018-05-31   
1      SO - 000102        Online  WARE-NMK1003   2017-12-31 2018-05-31   
2      SO - 000103   Distributor  WARE-UHY1004   2017-12-31 2018-05-31   
3      SO - 000104     Wholesale  WARE-NMK1003   2017-12-31 2018-05-31   
4      SO - 000105   Distributor  WARE-NMK1003   2018-04-10 2018-05-31   
...            ...           ...           ...          ...        ...   
7986  SO - 0008087      In-Store  WARE-MKL1006   2020-09-26 2020-12-30   
7987  SO - 0008088        Online  WARE-NMK1003   2020-09-26 2020-12-30   
7988  SO - 0008089        Online  WARE-UHY1004   2020-09-26 2020-12-30   
7989  SO - 0008090        Online  WARE-NMK1003   2020-09-26 2020-12-30   
7990  SO - 0008091      In-Store  WARE-UHY1004   2020-09-26 2020-12-30   

       ShipDate DeliveryDate CurrencyCode  _SalesTeamID  _CustomerID  ...  \
0    2018-06-14   2018-06-19      

3. Agrega las siguientes columnas a df_ base:

hint: Asegúrate de que las columnas sean de tipo datetime

In [29]:
#primero nos aseguramos que las columnas sean de tipo datetime
df_base['OrderDate'] = pd.to_datetime(df_base['OrderDate'])
df_base['ProcuredDate'] = pd.to_datetime(df_base['ProcuredDate'])
df_base['ShipDate'] =   pd.to_datetime(df_base['ShipDate'])
df_base['DeliveryDate'] = pd.to_datetime(df_base['DeliveryDate'])

# Agregar las columnas solicitadas
df_base['ProcurementDays'] = (df_base['OrderDate'] - df_base['ProcuredDate']).dt.days
df_base['ShippingDays'] = (df_base['ShipDate'] - df_base['OrderDate']).dt.days
df_base['DeliveryDays'] = (df_base['DeliveryDate'] - df_base['ShipDate']).dt.days
df_base['CustomerDays'] = df_base['ShippingDays'] + df_base['DeliveryDays']

# validamos el formato datetime
print(df_base[['OrderDate', 'ProcuredDate', 'ShipDate', 'DeliveryDate']].head())

# Agregar un salto de línea
print("\n")

# revisamos los calculos realizados
print(df_base[['ProcurementDays', 'ShippingDays', 'DeliveryDays', 'CustomerDays']].head())


   OrderDate ProcuredDate   ShipDate DeliveryDate
0 2018-05-31   2017-12-31 2018-06-14   2018-06-19
1 2018-05-31   2017-12-31 2018-06-22   2018-07-02
2 2018-05-31   2017-12-31 2018-06-21   2018-07-01
3 2018-05-31   2017-12-31 2018-06-02   2018-06-07
4 2018-05-31   2018-04-10 2018-06-16   2018-06-26


   ProcurementDays  ShippingDays  DeliveryDays  CustomerDays
0              151            14             5            19
1              151            22            10            32
2              151            21            10            31
3              151             2             5             7
4               51            16            10            26


4. Agrega una nueva columna discreta al df_base, con el nombre CustomerDaysInterval
que clasifique los valores de la columna CustomerDays en los siguientes intervalos:

* 0 to 15 days
* 15 to 30 days
* 30 to 45 days
* 45 to 60 days
* 60 to 75 days
* 75 to 90 days


hint: Utiliza pd.cut con estas etiquetas de Intervalo. Puedes utilizar la propiedad `.dt.days` para convertir la columna CustomerDays a valores numéricos.

La propiedad `.dt.days` te permite obtener directamente la representación numérica de la diferencia en días a partir de un objeto DateInterval

In [30]:
#primero definimos intervalos
bins = [0, 15, 30, 45, 60, 75, 90]

#Despues definimos etiquetas correspondientes para cada intervalo
labels = ['0 to 15 days', '15 to 30 days', '30 to 45 days', '45 to 60 days', '60 to 75 days', '75 to 90 days']

# Creamos CustomerDaysInterval usando pd.cut()
df_base['CustomerDaysInterval'] = pd.cut(df_base['CustomerDays'], bins=bins, labels=labels, right=False)

print(df_base[['CustomerDays', 'CustomerDaysInterval']].head())

   CustomerDays CustomerDaysInterval
0            19        15 to 30 days
1            32        30 to 45 days
2            31        30 to 45 days
3             7         0 to 15 days
4            26        15 to 30 days


5. Utiliza una tabla pivote para mostrar el conteo de órdenes agrupando por Sales
Team en las filas y CustomerDaysInterval en las columnas.

Este reporte debe cuadrar con los datos de la pestaña Reporte1 del archivo reportes.xlsx

In [32]:
#Primero creamos la tabla pivote con el conteo de órdenes, agrupado por 'Sales Team' y 'CustomerDaysInterval'
pivot_table = pd.pivot_table(df_base,
                             values='OrderNumber',
                             index='Sales Team',
                             columns='CustomerDaysInterval',
                             aggfunc='count',
                             fill_value=0)

print(pivot_table)


CustomerDaysInterval  0 to 15 days  15 to 30 days  30 to 45 days  \
Sales Team                                                         
Adam Hernandez                  71            176             55   
Anthony Berry                   69            176             53   
Anthony Torres                  70            160             58   
Carl Nguyen                     85            184             45   
Carlos Miller                   56            145             46   
Chris Armstrong                 86            152             54   
Donald Reynolds                 87            167             42   
Douglas Tucker                  76            148             53   
Frank Brown                     71            156             48   
George Lewis                    92            169             54   
Jerry Green                     81            162             53   
Joe Price                       80            129             57   
Jonathan Hawkins                74            14

6. Agrega la siguiente columna calculada a la base:


KPI Cálculo
GrossMargin = OrderQuantity*(UnitPrice*(1-Discount Applied)-UnitCost)

Visión de Negocio: El margen bruto es un indicador financiero que muestra la diferencia entrelos ingresos generados por la venta de un producto o servicio y los costos directos asociados con su producción o adquisición.

In [39]:
# Calcular margen bruto
df_base['GrossMargin'] = df_base['Order Quantity'] * ((df_base['Unit Price'] * (1 - df_base['Discount Applied'])) - df_base['Unit Cost'])

#print(df_base.head()) para validar info completa

#Realizamos el llamado a las columnas de importancia para el ejercicio
print(df_base[['Order Quantity', 'Unit Price', 'Discount Applied', 'Unit Cost', 'GrossMargin']].head())



   Order Quantity  Unit Price  Discount Applied  Unit Cost  GrossMargin
0               5      1963.1             0.075   1001.181    4073.4325
1               3      3939.6             0.075   3348.660     886.4100
2               1      1775.5             0.050    781.220     905.5050
3               8      2324.9             0.075   1464.687    5486.7640
4               8      1822.4             0.100   1476.144    1312.1280


7. En esta parte debes aplicar un porcentaje a la columna GrossMargin, pero ese
porcentaje dependerá del intervalo en que se encuentre GrossMargin.

KPI Cálculo = **CommissionsPercentage** % de comisión según la tabla de comisiones

hint: Utiliza pd.cut con labels numéricas para que la columna resultante sea de tipo float.

In [44]:
#Primero veremos un margen GrossMargin
bins = [0, 100, 1000, 10000, 100000]

#Despues definimos etiquetas con los porcentajes de comisión, esto lo realizaremos con float
labels = [0.05, 0.10, 0.15, 0.20]

#CommissionsPercentage usando pd.cut
df_base['CommissionsPercentage'] = pd.cut(df_base['GrossMargin'], bins=bins, labels=labels, right=False)

#aplicamos astype para asegurarnos del resultado
df_base['CommissionsPercentage'] = df_base['CommissionsPercentage'].astype(float)

print(df_base[['GrossMargin', 'CommissionsPercentage']].head())

   GrossMargin  CommissionsPercentage
0    4073.4325                   0.15
1     886.4100                   0.10
2     905.5050                   0.10
3    5486.7640                   0.15
4    1312.1280                   0.15


8. Calcula el monto de la comisión, utilizando la Tabla del Anexo.

KPI Cálculo
CommissionsAmount

GrossMargin*CommissionsPercentage

Intervalo Comisión
* GrossMargin entre $0 y $100 5%*  
* GrossMargin entre $100 y $1000 10%
* GrossMargin entre $1000 y $10000 15%
* GrossMargin entre $10000 y $100000 20%


In [45]:
#Realizamos la formula que nos muestran en la pregunta 8, basicamente una multiplicacion de las columnas ya trabajadas dado que ya tomamos en consideracion la tabla anexa de comision
df_base['CommissionsAmount'] = df_base['GrossMargin'] * df_base['CommissionsPercentage']

#imprimimos
print(df_base[['GrossMargin', 'CommissionsPercentage', 'CommissionsAmount']].head())


   GrossMargin  CommissionsPercentage  CommissionsAmount
0    4073.4325                   0.15         611.014875
1     886.4100                   0.10          88.641000
2     905.5050                   0.10          90.550500
3    5486.7640                   0.15         823.014600
4    1312.1280                   0.15         196.819200


9. Calcula la comisión sobre el margen bruto.

KPI Cálculo
NetMargin

GrossMargin-CommissionsAmount


In [59]:
#Realizamos la formula que nos muestran en la pregunta 8, basicamente una multiplicacion de las columnas ya trabajadas
df_base['NetMargin'] = df_base['GrossMargin'] - df_base['CommissionsAmount']

#imprimimos los 3 valores para validar
print(df_base[['GrossMargin', 'CommissionsAmount', 'NetMargin']].head())


   GrossMargin  CommissionsAmount    NetMargin
0    4073.4325         611.014875  3462.417625
1     886.4100          88.641000   797.769000
2     905.5050          90.550500   814.954500
3    5486.7640         823.014600  4663.749400
4    1312.1280         196.819200  1115.308800


In [50]:
print(df_base[['GrossMargin', 'CommissionsAmount','NetMargin','CommissionsPercentage']].head(10))

#validacion simple 4073.4325 − 611.014875 = 3462.417625 == al 0.15 %

   GrossMargin  CommissionsAmount    NetMargin  CommissionsPercentage
0    4073.4325         611.014875  3462.417625                   0.15
1     886.4100          88.641000   797.769000                   0.10
2     905.5050          90.550500   814.954500                   0.10
3    5486.7640         823.014600  4663.749400                   0.15
4    1312.1280         196.819200  1115.308800                   0.15
5    2700.1000         405.015000  2295.085000                   0.15
6    1908.1600         286.224000  1621.936000                   0.15
7     998.6350          99.863500   898.771500                   0.10
8    2017.2360         302.585400  1714.650600                   0.15
9    5164.8960         774.734400  4390.161600                   0.15


10. Utilizando una tabla pivote, muestra la suma de los valores GrossMargin, NetMargin y CommissionAmount agrupando por Sales Team en las filas.

Este reporte debe cuadrar con los datos de la pestaña Reporte2 del archivo reportes.xlsx

Intervalo Comisión
* GrossMargin entre $0 y $100 5%*  
* GrossMargin entre $100 y $1000 10%
* GrossMargin entre $1000 y $10000 15%
* GrossMargin entre $10000 y $100000 20%


In [55]:
#Creamos tabla pivot con los tres valores anteriores y sumandolos
pivot_table_sales_team = pd.pivot_table(df_base,
                             values=['GrossMargin', 'NetMargin', 'CommissionsAmount'],
                             index='Sales Team',
                             aggfunc='sum',
                             fill_value=0)

#imprimimos la tabla
print(pivot_table_sales_team)

#quizas una sugerencia al negocio es agregar el porcentaje (CommissionsPercentage)

                     CommissionsAmount  GrossMargin      NetMargin
Sales Team                                                        
Adam Hernandez           126637.207525  779095.1955  677987.332975
Anthony Berry            124061.181475  767943.9500  654798.341525
Anthony Torres           117962.211675  743901.4355  636758.048825
Carl Nguyen              123843.212050  774781.4675  661874.732450
Carlos Miller            110882.093875  668752.9290  580326.019125
Chris Armstrong          122565.707975  781608.7005  660857.419525
Donald Reynolds          143768.338700  885386.8095  754038.260800
Douglas Tucker           120806.403550  745528.5640  633786.054450
Frank Brown              104839.466050  670881.2510  570076.792950
George Lewis             136852.255325  839150.7460  710824.776675
Jerry Green              126660.436425  788587.7555  670379.034075
Joe Price                128822.930100  787707.4425  662662.106400
Jonathan Hawkins         126216.945000  781391.5870  664629.88